This Notebook was built with the purpose of understanding how to compute gradients and optimization routines with Graph Nets library. The compatibility between keras, tensorflow and Graph Nets is not fully supported. There are some grey areas which make all the standard implementation fails. Specifically, it is needed among other things to use the Adam optimizer of the Sonnet module. This was only discovered thanks to the notebook from the demo of graph nets 2 (the only example compatible with tensorflow2) called sort.py at the link:
Minor adjustment in the source code are needed to run the notebook sort.ipynb. 
The pipeline for this notebook was: 
0) Understanding how the custom gradient implementation of tensorflow behaves
1) taking the most naive version from the implementation of the main code
2) Comparing step by step the sort.ipynb notebook with the structure needed for our work
3) Finding out how to compute gradients (None values were ubiquitos initially)
4) subsequently understanding how to apply the optimizer to this naive version 
5) Increasing the complexity in order to implement IT-SWO 
    Point 5 can be subsequently divided in subtasks:
        5.1)
        5.2)
        5.3)


## Step 0: 
Of course there are plenty of other examples, this is just a representative one for comparison.


In [17]:
import tensorflow as tf
@tf.custom_gradient
def custom_relu2(x):
    def relu_grad(dy):
        grad=  tf.cast( x > 0, dtype=tf.float32)*dy
        return grad
    return tf.maximum(x, 0.), relu_grad
data = tf.Variable([0.5, -0.2, 0.0, -2.5, 3.0], dtype=tf.float32)


In [18]:
import tensorflow as tf

@tf.custom_gradient
def custom_relu(x):
    def relu_forward(x):
        return tf.maximum(x, 0)
    
    def relu_grad(dy):
        grad = tf.cast(x > 0, dtype=tf.float32) * dy
        return grad

    return relu_forward(x), relu_grad

# Define a trainable variable
w = tf.Variable([1.0], dtype=tf.float32)  # A simple weight variable

# Define an input
x = tf.constant([1.0, -1.0, -2.0, 2.0, 3.0], dtype=tf.float32)

# Define an optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)

for step in range(3):
    with tf.GradientTape() as tape:
        y = custom_relu(w * x)  # Apply ReLU to a linear transformation of x
        print(y, type(y))
        # Compute gradients of y with respect to w
        gradients = tape.gradient(y, w)
        print(gradients)
        # Apply gradients to update w
        optimizer.apply_gradients(zip([gradients], [w]))

        print("Updated w:", w.numpy())



tf.Tensor([1. 0. 0. 2. 3.], shape=(5,), dtype=float32) <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([6.], shape=(1,), dtype=float32)
Updated w: [0.94]
tf.Tensor([0.94 0.   0.   1.88 2.82], shape=(5,), dtype=float32) <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([6.], shape=(1,), dtype=float32)
Updated w: [0.88]
tf.Tensor([0.88 0.   0.   1.76 2.64], shape=(5,), dtype=float32) <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([6.], shape=(1,), dtype=float32)
Updated w: [0.82]


## Step 1 




In [19]:
import tensorflow as tf
import sonnet as snt
from graph_nets import modules

hidden_layer_size=4   #This will be 128
output_emb_size=4      #This has to be 64 at the end
# Define the MLP model
class MLPModel_glob(snt.Module):
    def __init__(self, name=None):
        super(MLPModel_glob, self).__init__(name=name)
        self.layer1 = snt.Linear(output_size=hidden_layer_size, name='Glob_layer')
       

    def __call__(self, inputs):
        out = tf.nn.relu(self.layer1(inputs))
        
        return out
class MLPModel_enc(snt.Module):
    def __init__(self, name=None):
        super(MLPModel_enc, self).__init__(name=name)
        self.layer1 = snt.Linear(output_size=hidden_layer_size, name='enc_layer')
        

    def __call__(self, inputs):
        out = tf.nn.relu(self.layer1(inputs))

        return out

# Define the Encoder layer
class Encoder(modules.GraphNetwork):
    def __init__(self):
        super(Encoder, self).__init__(
            edge_model_fn=MLPModel_enc,
            node_model_fn=MLPModel_enc,
            global_model_fn=MLPModel_glob
        )

    def __call__(self, inputs):
        return super(Encoder, self).__call__(inputs)

    
class MLPModel_dec(snt.Module):
    def __init__(self, name=None):
        super(MLPModel_dec, self).__init__(name=name)
        self.layer1 = snt.Linear(output_size=hidden_layer_size, name='dec_layer')

    def __call__(self, inputs):
        out = tf.nn.relu(self.layer1(inputs))

        return out

class Decoder(modules.GraphNetwork):
    def __init__(self):
        super(Decoder, self).__init__(
            edge_model_fn=MLPModel_dec,
            node_model_fn=MLPModel_dec,
            global_model_fn=MLPModel_glob
        )

    def __call__(self, inputs):
        return super(Decoder, self).__call__(inputs)    
class PoolingLayer_double(tf.Module):
    def __init__(self):
        super(PoolingLayer_double, self).__init__()
        self.linear = snt.Linear(output_size=2, name='linear_pool')
        self.global_transform = snt.Linear(output_size=2, name='global_transform')

    def __call__(self, inputs):
        # Sum-pooling over nodes and edges
        pooled_nodes = tf.reduce_sum(inputs.nodes, axis=0)
        pooled_edges = tf.reduce_sum(inputs.edges, axis=0)
        pooled_features = tf.concat([pooled_nodes, pooled_edges], axis=0)
        
        transformed = self.linear(tf.expand_dims(pooled_features, axis=0))
        
        # Transform globals to match the shape of transformed
        transformed_globals = self.global_transform(0.05 *inputs.globals)
        #### THIS IS THE MOST RELEVANT PART, why again I can not use elu here? Is something related to the metric as well
        out = tf.nn.elu(transformed + transformed_globals)
        
        return out    
class GNN_double_output(snt.Module):
    def __init__(self):
        super(GNN_double_output, self).__init__()
        self.encoder = Encoder()
        self.pooling_layer = PoolingLayer_double()

    def __call__(self, inputs):
        encoded = self.encoder(inputs)

        output = self.pooling_layer(encoded)
        return output

In [20]:
import numpy as np
import time
import networkx as nx
from graph_nets import utils_np, utils_tf

# Set a random seed
start=time.time()
SEED = 1
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Define the lattice size
lattice_size = (2,2)
#Batch size, the number of graphs that will be computed simultaneously by the GNN
batch_size= 4

# Create a square lattice
G = nx.grid_2d_graph(*lattice_size, periodic=True)
# Number of sites
num_sites = lattice_size[0] * lattice_size[1]

# Relabel the nodes to use integers
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)
# Initialize the sublattice encoding
sublattice_encoding = np.zeros((num_sites, 2))  # Two sublattices
sublattice_encoding[::2, 0] = 1  # Sublattice 1
sublattice_encoding[1::2, 1] = -1  # Sublattice 2
# Create a dictionary where the keys are the node indices and the values are dictionaries
# containing the 'features' field and the corresponding sublattice encoding
node_dict = {i: {"features": sublattice_encoding[i]} for i in range(num_sites)}
# Use the dictionary to set the node attributes in the graph
#nx.set_node_attributes(G, node_dict)
# Add 'features' to nodes
for node in G.nodes():
    G.nodes[node]['features'] = sublattice_encoding[node]
# Add 'features' to edges
# Add 'features' to edges
for edge in G.edges():
    u, v = edge
    G.edges[u, v]['features'] = [1.0]  # Replace with your actual edge features
    G.edges[v, u]['features'] = [1.0]  # Add undirected edge # Replace with your actual edge features
# Now convert the networkx graph to a GraphsTuple
graph_tuple = utils_np.networkxs_to_graphs_tuple([G])

# Number of configurations to generate
n_configs = 6
n_graph=n_configs*batch_size
# Generate the basis configurations
basis_configs = np.random.randint(2, size=(n_graph, num_sites)) * 2 - 1  # Random spins (-1 or 1)

# Concatenate the basis configurations and the sublattice encoding to form the node features
node_features = np.concatenate([basis_configs[:, :, np.newaxis], np.repeat(sublattice_encoding[np.newaxis, :, :], n_graph, axis=0)], axis=2)

# Get the edge indices
edge_index = np.array(G.edges()).T
edge_index_duplicated = np.concatenate([edge_index, edge_index[::-1]], axis=1)
bias_value=0.05
# Create a list of graph dicts
graph_tuples = []
for i in range(n_graph):
    graph_dict = {
        'globals': np.array([0.05]),
        'nodes': node_features[i],
        'edges': np.full((edge_index_duplicated.shape[1], 1), bias_value),
        'senders': edge_index_duplicated[0],
        'receivers': edge_index_duplicated[1]
    }
    
    # Convert to a GraphsTuple and append to the list
    graph_tuples.append(utils_tf.data_dicts_to_graphs_tuple([graph_dict]))


print("end time:", time.time()-start)

end time: 0.16258454322814941


In [21]:
# Instantiate the model
simple_gnn = GNN_double_output()
for i in range(1):
    a,b =simple_gnn(graph_tuples[i])[0]
    print(a,b)

tf.Tensor(1.8287344387280773, shape=(), dtype=float64) tf.Tensor(-0.5772058972920167, shape=(), dtype=float64)


In [22]:
def mock_loss_function(amplitude, phase):
    """A mock loss function for illustration purposes h"""
    return tf.abs(1- phase*amplitude**2)


## Step 2 to 4
Most of the work here was done by trial and error and a lot of print statements among other strategies. 
The end result looks decently clean but the hidden work behind was quite intensive.
Observations: sonnet modules seems to not be compatible with keras optimizers built on tensorflow. We need to use the sonnet optimizers.
Furthermore, the sonnet optimizer does not allow for a learning schedule with Exponential decay as was intended to be used initially. 
The only useful implementation is of the standard Adam optimization routine. (Trivial SGD is inadequate for so many parameters, ca va sans dire)
No fancy version of Adam are available, maybe it is for the best though.

In [23]:
import sonnet as snt
initial_learning_rate = 7e-3
decay_steps = 8 * 1e5
decay_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate, staircase=True)
print(lr_schedule)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.99, clipnorm=1)


In [24]:
initial_learning_rate = 7e-3
optimizer = snt.optimizers.Adam(initial_learning_rate,0.9)

def train_step(model, input_graph, optimizer):

    with tf.GradientTape() as tape:
        output = model(input_graph)[0]

        tape.watch(model.trainable_variables)
        print("output: \n", output)
        amplitude, phase = output
        print(amplitude, phase, "types", type(amplitude), type(phase))
            
        loss = mock_loss_function(amplitude,phase)
            
    #print("model variables: \n",model.trainable_variables) -< those are fine and are all tf.variables
    print("Is it lossing: \n", loss, type(loss))
    gradients = tape.gradient(loss, model.trainable_variables)
    #print("are model variables and gradients two lists?", type(model.trainable_variables), type(gradients))

    #print("model gradients: \n", gradients)
    #for var, grad in zip(model.trainable_variables, gradients):
    #    print(f"{var.name}: Gradient {'is None' if grad is None else 'exists'}")
    optimizer.apply(gradients, model.trainable_variables)
    
    return output, loss

np.set_printoptions(precision=2)
# Training loop
for step in range(1):
    #print(graph_tuples[0])
    outputs, loss = train_step(simple_gnn, graph_tuples[0], optimizer)
    if step % 1 == 0:
        print(f"Step {step}, Loss: {loss.numpy()}")


output: 
 tf.Tensor([ 1.83 -0.58], shape=(2,), dtype=float64)
tf.Tensor(1.8287344387280773, shape=(), dtype=float64) tf.Tensor(-0.5772058972920167, shape=(), dtype=float64) types <class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>
Is it lossing: 
 tf.Tensor(2.9303321626082566, shape=(), dtype=float64) <class 'tensorflow.python.framework.ops.EagerTensor'>
Step 0, Loss: 2.9303321626082566


## Step 5
### 5.1
In this section of the notebook we are going to implement the real loss function and attempt to train a simplified model of our GNN on a 2x2 square lattice. 
The following implementation will rely on scipy sparse.
This implementation is suboptimal and is done for educational purpose. The reason is that implementing a Sparse Matrix is resource intensive regardless. The key reasoning is that one does not need to fully implement the matrix, just vector matrix multiplications, and those follow some specific patterns due to ortoghonality, plase look at step 5.2 for an implementation that does not need explicitly the matrix.



In the following cell block we generate the input graph and initialize the edges and node values

In [25]:
import numpy as np
import time
import networkx as nx
from graph_nets import utils_np, utils_tf

start=time.time()
SEED = 1
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Define the lattice size
lattice_size = (2,2)
#Batch size, the number of graphs that will be computed simultaneously by the GNN, here they are set to 4 because the 
#total number of states is 2**4, and any value above is equivalent to optimizing the GNN on the whole hilbert space.
# We are interested in the main pipeline, and want to generalize the network to larger scales thereafter
batch_size= 4

# Create a square lattice with periodic boundary conditions. This is expecially useful in larger graphs where
# having periodic boundary conditions allows for a more decent approximation of an infinite lattice, for small size graphs
# periodic boundary conditions have a little effect.
G = nx.grid_2d_graph(*lattice_size, periodic=True)
# Number of sites
num_sites = lattice_size[0] * lattice_size[1]

# Relabel the nodes to use integers
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)
# Initialize the sublattice encoding
sublattice_encoding = np.zeros((num_sites, 2))  # Two sublattices
sublattice_encoding[::2, 0] = 1  # Sublattice 1
sublattice_encoding[1::2, 1] = -1  # Sublattice 2
# Create a dictionary where the keys are the node indices and the values are dictionaries
# containing the 'features' field and the corresponding sublattice encoding
node_dict = {i: {"features": sublattice_encoding[i]} for i in range(num_sites)}

# Add 'features' to nodes, in this case the feature is a one hot vectore that represents the sublattice encoding of each node
for node in G.nodes():
    G.nodes[node]['features'] = sublattice_encoding[node]
# Add 'features' to edges
# Add 'features' to edges
for edge in G.edges():
    u, v = edge
    G.edges[u, v]['features'] = [1.0]  # 
    G.edges[v, u]['features'] = [1.0]  # Add undirected edge 
# Now convert the networkx graph to a GraphsTuple
graph_tuple = utils_np.networkxs_to_graphs_tuple([G])

# Number of batches of configurations to generate, here is set to 4 because 4x4=2**4= Hilbert space, and we don't need more than that
n_configs = 4
n_graph=n_configs*batch_size
# Generate the basis configurations
basis_configs = np.random.randint(2, size=(n_graph, num_sites)) * 2 - 1  # Random spins (-1 or 1)

# Concatenate the basis configurations and the sublattice encoding to form the node features. I BELIEVE THIS WAY TO ENCODE THE NODE FEATURES IS NOT RIGHT
node_features = np.concatenate([basis_configs[:, :, np.newaxis], np.repeat(sublattice_encoding[np.newaxis, :, :], n_graph, axis=0)], axis=2)
#print(node_features)
# Get the edge indices
edge_index = np.array(G.edges()).T
edge_index_duplicated = np.concatenate([edge_index, edge_index[::-1]], axis=1)
#Initialization of edges, it should be 0, we set it to 0.05.
bias_value=0.05
# Create a list of graph dicts
graph_tuples = []
for i in range(n_graph):
    graph_dict = {
        'globals': np.array([0.05]),
        'nodes': node_features[i],
        'edges': np.full((edge_index_duplicated.shape[1], 1), bias_value),
        'senders': edge_index_duplicated[0],
        'receivers': edge_index_duplicated[1]
    }
    
    # Convert to a GraphsTuple and append to the list
    graph_tuples.append(utils_tf.data_dicts_to_graphs_tuple([graph_dict]))


print("end time:", time.time()-start)

end time: 0.10471224784851074


In [26]:
import matplotlib.pyplot as plt
from compgraph.gnn_src_code import GNN_double_output

from compgraph.sparse_ham import create_spin_operators, construct_sparse_hamiltonian, sites_to_sparse

configurations, value_list= sites_to_sparse(basis_configs)
# Uncomment the following line to see the distribution of the initial random configurations
#plt.plot(np.sort(value_list))
#Here we noticed that the second environment created is not compatible for some obscure reasons, probably qutip compatibility. We are gonna stick to this one for now. ASK patrick

J2=2.0
spin_operators=create_spin_operators(G)
Hamiltonian = construct_sparse_hamiltonian(G, spin_operators, J2)
less_trivial_gnn=GNN_double_output()

ImportError: cannot import name 'settings' from 'qutip.settings' (/home/stefanotroffa/miniconda3/envs/gnets2/lib/python3.12/site-packages/qutip/settings.py)

In [ ]:
import time
import numpy as np
import tensorflow as tf
#reload(compgraph.tensor_wave_functions)

from compgraph.sparse_ham import compute_wave_function_csr,  innerprod_sparse
#THE inner training step should work for one configuration at a time if logic has any ground on reality
np.set_printoptions(precision=2, suppress=True)
optimizer_snt = snt.optimizers.Adam(initial_learning_rate,0.9)

def evolving_function(wave):
    """This loss function is for illustrative purposes"""
    auxphi=tf.sparse.map_values(tf.multiply,wave, -0.05)

    phi=tf.sparse.map_values(tf.add,wave,auxphi)
    psi_conj= tf.sparse.map_values(tf.math.conj, wave)
    prod=tf.sparse.map_values(tf.multiply,psi_conj,phi)

    return prod.values[:]
def wave_function(model, graph_batch, graph_batch_indices):
    unique_data = {}  # Dictionary to store unique indices and their corresponding values
    size=2**len(graph_batch[0].nodes)
    # Compute the wave function components for each graph tuple
    for idx, graph_tuple in enumerate(graph_batch):
        #print(graph_batch_indices, type(graph_batch_indices))
        # Extract the row index from the configuration
        row_index = graph_batch_indices[idx].indices[0]
        # Check if the index is already in the dictionary
        if row_index in unique_data:
            pass  # Sum up the values for repeated indices
        else:
            amplitude, phase = model(graph_tuple)[0]
            # Convert amplitude to complex tensor with zero imaginary part
            complex_coefficient=tf.complex(real=amplitude, imag=phase)

        
            unique_data[row_index] = complex_coefficient  # Add new index and value to the dictionary
    
    # Convert dictionary to lists
    values = list(unique_data.values())
    indices = [[key, 0] for key in unique_data.keys()]
    
    # Convert lists to tensors
    values_tensor = tf.stack(values, axis=0)
    indices_tensor = tf.constant(indices, dtype=tf.int64)
    
    # Create a sparse tensor
    sparse_tensor = tf.sparse.SparseTensor(indices=indices_tensor, values=values_tensor, dense_shape=[size, 1])
    
    return sparse_tensor




def sparse_rep_inner_training_step(model, graph_batch, graph_batch_indices, target_phi, optimizer):

    with tf.GradientTape() as tape:
        output = wave_function(model, graph_batch, graph_batch_indices)

        tape.watch(model.trainable_variables)
        print("output: \n", output)
            
        loss = evolving_function(output)
            
    #print("model variables: \n",model.trainable_variables) -< those are fine and are all tf.variables
    print("Is it lossing: \n", loss, type(loss))
    gradients = tape.gradient(loss, model.trainable_variables)
    #print("are model variables and gradients two lists?", type(model.trainable_variables), type(gradients))

    #print("model gradients: \n", gradients)
    #for var, grad in zip(model.trainable_variables, gradients):
    #    print(f"{var.name}: Gradient {'is None' if grad is None else 'exists'}")
    optimizer_snt.apply(gradients, model.trainable_variables)
    
    return output, loss


start=0
for step in range(1):  # IT-SWO steps
    # Compute phi once at the beginning of each outer step, this is the ITO of psi
    graph_tuples_batch=graph_tuples[start:start + batch_size]
    graph_tuples_batch_indices= configurations[start:start + batch_size]
    psi_csr = compute_wave_function_csr(graph_tuples_batch, less_trivial_gnn, graph_tuples_batch_indices)
    
    beta = 0.05 #This parameter determines the amount of imaginary time evolution at each outer step
    phi_csr = psi_csr - beta * Hamiltonian.dot(psi_csr)
    print("WE", innerprod_sparse(phi_csr, psi_csr))
    for innerstep in range(6):  # Inner loop iterations: here we let psi approximate its ITO phi

        outputs, loss = sparse_rep_inner_training_step(less_trivial_gnn, graph_tuples_batch, graph_tuples_batch_indices, phi_csr, optimizer)
       
        print(outputs,loss)
        if step % 1 == 0:
            print(f"Step {step}, Loss: {loss.numpy()}")
        
        #print(gradients[0][0], "step", step)
        
    # Update the start index for the next batch
    start += batch_size


WE   (0, 0)	(0.3125555033502897+3.469446951953614e-18j)
output: 
 SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([-0.08+0.28j  0.15+0.16j -0.1 +0.31j -0.1 +0.31j], shape=(4,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64))
Is it lossing: 
 tf.Tensor([0.08+0.j 0.05+0.j 0.1 +0.j 0.1 +0.j], shape=(4,), dtype=complex128) <class 'tensorflow.python.framework.ops.EagerTensor'>
SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([-0.08+0.28j  0.15+0.16j -0.1 +0.31j -0.1 +0.31j], shape=(4,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64)) tf.Tensor([0.08+0.j 0.05+0.j 0.1 +0.j 0.1 +0.j], shape=(4,), dtype=complex128)
Step 0, Loss: [0.08+0.j 0.05+0.j 0.1 +0.j 0.1 +0.j]
output: 
 SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([0.12-0.31j 0.14-0.3

In [ ]:
import compgraph

from compgraph.tensor_wave_functions import adjust_dtype_and_multiply, convert_csr_to_sparse_tensor

indexing= np.column_stack((Hamiltonian.nonzero()))
values_H= Hamiltonian.data
shape_H = Hamiltonian.shape
H_tensor= tf.sparse.SparseTensor(indexing,values_H,shape_H)
H_tensor = tf.sparse.reorder(H_tensor)
H2_tensor = convert_csr_to_sparse_tensor(Hamiltonian)
print(tf.assert_equal(H_tensor.values, H2_tensor.values))
print(H_tensor.values-H2_tensor.values)

None
tf.Tensor(
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j], shape=(56,), dtype=complex128)


In [ ]:
output=  wave_function(less_trivial_gnn, graph_tuples_batch, graph_tuples_batch_indices)
output_csr = compute_wave_function_csr(graph_tuples_batch, less_trivial_gnn, graph_tuples_batch_indices)
auxphi=tf.sparse.map_values(tf.multiply,output, -0.05)
auxphi2=tf.sparse.map_values(tf.add,output,auxphi)
prod=tf.sparse.map_values(tf.multiply,output,auxphi2)
print(auxphi, auxphi2,prod) 
print(prod.values[:])
conj_output = tf.sparse.map_values(tf.math.conj, output)
prod=tf.sparse.map_values(tf.multiply,conj_output,output)
print(prod.values[:])


SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([-0.01-0.01j -0.01-0.01j -0.01-0.01j -0.01-0.01j], shape=(4,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64)) SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([0.1 +0.1j  0.13+0.15j 0.1 +0.11j 0.1 +0.11j], shape=(4,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64)) SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([-0.  +0.02j -0.01+0.04j -0.  +0.02j -0.  +0.02j], shape=(4,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64))
tf.Tensor([-0.  +0.02j -0.01+0.04j -0.  +0.02j -0.  +0.02j], shape=(4,), dtype=complex128)
tf.Tensor([0.02+0.j 0.04+0.j 0.02+0.j 0.02+0.j], shape=(4,), dtype=complex128)


In [ ]:

from scipy.sparse import csr_matrix
csr
phi= adjust_dtype_and_multiply(H_tensor,output)

print(tf.sparse.reorder(output).indices)
print(phi)

tf.Tensor(
[[ 0  0]
 [ 6  0]
 [ 9  0]
 [12  0]], shape=(4, 2), dtype=int64)
SparseTensor(indices=tf.Tensor(
[[ 0  0]
 [ 3  0]
 [ 5  0]
 [ 6  0]
 [ 9  0]
 [10  0]
 [12  0]], shape=(7, 2), dtype=int64), values=tf.Tensor(
[ 0.51+0.54j  0.12+0.13j  0.56+0.58j  0.46+0.52j  0.39+0.39j  0.56+0.58j
 -0.32-0.31j], shape=(7,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64))


In [ ]:
phi_csr = Hamiltonian.dot(output_csr)
phi_sparse_coo = phi_csr.tocoo()
indices = np.column_stack((phi_sparse_coo.row, phi_sparse_coo.col))
phi_sparse_tf = tf.cast(tf.sparse.SparseTensor(indices, phi_sparse_coo.data, phi_sparse_coo.shape), dtype=tf.complex128)
print(phi_sparse_tf)

SparseTensor(indices=tf.Tensor(
[[ 0  0]
 [ 3  0]
 [ 5  0]
 [ 6  0]
 [ 9  0]
 [10  0]
 [12  0]], shape=(7, 2), dtype=int64), values=tf.Tensor(
[ 0.68+0.78j  0.11+0.11j  0.52+0.54j  0.38+0.39j  0.38+0.39j  0.52+0.54j
 -0.3 -0.32j], shape=(7,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64))


In [ ]:
print(output, '\n',output_csr)

SparseTensor(indices=tf.Tensor(
[[12  0]
 [ 0  0]
 [ 6  0]
 [ 9  0]], shape=(4, 2), dtype=int64), values=tf.Tensor([0.1 +0.11j 0.14+0.16j 0.11+0.11j 0.11+0.11j], shape=(4,), dtype=complex128), dense_shape=tf.Tensor([16  1], shape=(2,), dtype=int64)) 
   (0, 0)	(0.13569815+0.15544538j)
  (6, 0)	(0.11025918+0.111020915j)
  (9, 0)	(0.11025918+0.111020915j)
  (12, 0)	(0.101626776+0.1082612j)
